# 0 Imports

In [1]:
import os, warnings
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
import seaborn as sns
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image_dataset_from_directory

def set_seed(seed=31415):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
set_seed()

# Set Matplotlib defaults
plt.rc('figure', autolayout=True)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=18, titlepad=10)
plt.rc('image', cmap='magma')
warnings.filterwarnings("ignore") # to clean up output cells

***
# 1 Custom CNN

In [12]:
ds_train_ = image_dataset_from_directory(
    "data/Images",
    labels="inferred",
    label_mode="categorical",
    image_size=[128, 128],
    interpolation="nearest",
    batch_size=32,
    seed=0,
    shuffle=True,
    validation_split=0.8,
    subset="training"
)


ds_valid_ = image_dataset_from_directory(
    "data/Images",
    labels="inferred",
    label_mode="categorical",
    image_size=[128, 128],
    interpolation="nearest",
    batch_size=32,
    seed=0,
    shuffle=True,
    validation_split=0.2,
    subset="validation"
)


def convert_to_float(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE
ds_train = (
    ds_train_
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)
ds_valid = (
    ds_valid_
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)

Found 20580 files belonging to 120 classes.
Using 4116 files for training.
Found 20580 files belonging to 120 classes.
Using 4116 files for validation.


In [21]:
model = keras.Sequential([
    # base
    layers.Conv2D(
        input_shape=[128, 128, 3],
        filters=32,
        kernel_size=3,
        activation="relu",
        padding="same"
        ),
    layers.MaxPool2D(),
    
    # head
    layers.Flatten(),
    layers.Dense(units=6, activation="relu"),
    layers.Dense(units=1, activation="sigmoid")
])

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 128, 128, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 32)       0         
 )                                                               
                                                                 
 flatten_4 (Flatten)         (None, 131072)            0         
                                                                 
 dense_4 (Dense)             (None, 6)                 786438    
                                                                 
 dense_5 (Dense)             (None, 1)                 7         
                                                                 
Total params: 787,341
Trainable params: 787,341
Non-trainable params: 0
________________________________________________

In [23]:
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics="categorical_accuracy"
)

history = model.fit(
    ds_train,
    validation_data=ds_valid,
    epochs=1,
    verbose=0
)

ValueError: in user code:

    File "/home/alex/.local/lib/python3.8/site-packages/keras/engine/training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "/home/alex/.local/lib/python3.8/site-packages/keras/engine/training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/alex/.local/lib/python3.8/site-packages/keras/engine/training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "/home/alex/.local/lib/python3.8/site-packages/keras/engine/training.py", line 809, in train_step
        loss = self.compiled_loss(
    File "/home/alex/.local/lib/python3.8/site-packages/keras/engine/compile_utils.py", line 201, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "/home/alex/.local/lib/python3.8/site-packages/keras/losses.py", line 141, in __call__
        losses = call_fn(y_true, y_pred)
    File "/home/alex/.local/lib

In [ ]:
history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ("loss", "val_loss")].plot()
history_frame.loc[:, ("binary_accuracy", "val_binary_accuracy")].plot()

***
# 2 Transfer learning CNN